# Data structures for single-cell and multi-modal data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/flcello/PDC2024Stegle/blob/master/01_data_structures.ipynb)

In this exercise, we'll get familiar with the AnnData and MuData frameworks for working with single-cell and multi-modal data in Python.
These frameworks make working with data much more convenient compared to using for example plain Numpy arrays or Pandas DataFrames. In the last part of the exercise, we'll process some actual data using these frameworks.

In [ ]:
# !pip install muon==0.1.5 mudatasets mofapy2 mofax leidenalg seaborn==0.12.2 "numpy<2.0" anndata==0.10.2

## AnnData

You can think of an AnnData object as a data matrix of observations x variables with additional metadata. Visit the [documentation](https://anndata.readthedocs.io/en/latest/index.html) for a [quick introduction](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html)

To demonstrate basic usage, we first import relevant Python packages and then create an AnnData object with random data.

In [27]:
counts

array([[0, 0, 0, ..., 1, 1, 3],
       [1, 0, 0, ..., 0, 2, 1],
       [2, 1, 0, ..., 4, 0, 1],
       ...,
       [1, 0, 1, ..., 0, 1, 1],
       [2, 2, 0, ..., 2, 1, 2],
       [2, 0, 1, ..., 1, 1, 1]])

In [ ]:
import anndata
import numpy as np

# Simulate count matrix with 100 rows (observations) and 2000 columns (features)
counts = np.random.poisson(1, size=(100, 2000))
adata = anndata.AnnData(counts)
adata

AnnData object with n_obs × n_vars = 100 × 2000

The .X attribute contains the data matrix.

In [28]:
adata.X

array([[0, 0, 0, ..., 1, 1, 3],
       [1, 0, 0, ..., 0, 2, 1],
       [2, 1, 0, ..., 4, 0, 1],
       ...,
       [1, 0, 1, ..., 0, 1, 1],
       [2, 2, 0, ..., 2, 1, 2],
       [2, 0, 1, ..., 1, 1, 1]])

For very large and sparse data matrices (many zeros), it's recommended to convert the numpy array to a sparse matrix object first, this is more memory efficient.

In [ ]:
from scipy.sparse import csr_matrix

# convert counts to a sparse matrix
counts_sparse = csr_matrix(counts)
adata_sparse = anndata.AnnData(counts_sparse)
adata_sparse

Now the .X attribute is a sparse matrix.

In [29]:
adata_sparse.X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 126636 stored elements and shape (100, 2000)>

Our AnnData object does not have any metadata yet. We can assign names to observations (cells) and variables (genes).

In [32]:
adata.obs_names = [f"Cell_{i}" for i in range(adata.n_obs)]
adata.var_names = [f"Gene_{i}" for i in range(adata.n_vars)]
adata

AnnData object with n_obs × n_vars = 100 × 2000

Now we can subset the AnnData object using cell and gene names.

In [35]:
adata[["Cell_1", "Cell_10"], ["Gene_5", "Gene_1900"]].X

ArrayView([[0, 0],
           [1, 2]])

Note that we obtained a view of an AnnData object. The view does not store any data but points to the original AnnData object instead to save memory. The view will automatically convert itself to a full AnnData object by copying the relevant subset of the original object if we try to add some data to the view.

We can also add some metadata describing the observations. To demonstrate this, we randomly assign one of three random cell types to a column in the `.obs` attribute which is a Pandas DataFrame.

In [ ]:
adata.obs["cell_type"] = np.random.choice(["B", "T", "Monocyte"], size=(adata.n_obs,))
adata.obs

Note that the description of the AnnData object now includes the new column.

In [ ]:
adata

We can also subset the AnnData object using boolean indexing. For example, we can get all B cells.

In [ ]:
bdata = adata[adata.obs["cell_type"] == "B"]
bdata

AnnData can store multiple data matrices in the `.layers` attribute. This is useful if one wants to keep both original and transformed, e.g. normalized, data. All matrices must have the same shape as the original `.X`.

In [ ]:
adata.layers["log_transformed"] = np.log1p(adata.X)
adata

### Exercise
Try to add a new layer called `counts` to the AnnData object with the reverse transformation, `np.expm1()`.

In [ ]:
# Your solution here
# ...


Inspect the resulting matrix. Is it different to the original values in `.X`?

In [ ]:
# Your solution here
# ...

Now repeat the transformation steps with the `adata_sparse` object. To display the values of a sparse matrix, you can use the `.toarray()` method.

## MuData

MuData objects contain a dictionary of AnnData objects and are used for multimodal data. AnnData objects within a MuData container are aligned and can be jointly subsetted.

To demonstrate this, we will use the `adata` and `bdata` objects from above. Since MuData assumes that variables are unique to each modality, we first change `bdata`'s variable names and then create a MuData object.

In [ ]:
import mudata
mudata.set_options()

# rename the variable names in bdata to "Protein_0", "Protein_1", ...
bdata.var_names = [f"Protein_{i}" for i in range(bdata.n_vars)]
bdata.var_names

In [ ]:
# create a multimodal MuData object from the two AnnData objects
mdata = mudata.MuData({"rna": adata, "prot": bdata})
mdata

We can access individual modalities using a dictionary-like interface.

In [ ]:
mdata["rna"]

We can subset the MuData object using integer or string based indexing.

In [ ]:
mdata[20:42, ["Gene_42", "Protein_42"]]

Note that less than the expected 22 cells from the `prot` modality are included in the subset. This is because we created the `prot` modality from only a subset of all cells, and MuData uses the cell names (`.obs_names`) to mach observations in different modalities to each other.

### Exercise

Pick two random Protein features and plot them against each other using a scatterplot.

In [ ]:
# Your solution here
# ...


Simulate missing measurements by placing `np.nan` in the first 10 cells of the "prot" modality.

In [ ]:
# Your solution here
# ...